In [1]:
import torch as t
from mcs import MCS
from utils import indep_chkboard_sets
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

In [2]:
t.autograd.set_detect_anomaly(True)
init_state = t.zeros(1,8,8)
init_state[0,4,4] += 1

checkerboard_sets = indep_chkboard_sets(init_state, tile_height=4, tile_width=4)
target_vol = 1.
temperature = t.tensor(27.)
temperature.requires_grad_()

tensor(27., requires_grad=True)

In [3]:
states = [init_state.detach().clone().squeeze().numpy()]
state = init_state.detach().clone()
for i in range(100):
    print(f"-------- MCS {i} --------------")
    state, steps = MCS(state, checkerboard_sets, target_vol, temperature)
    print(state)
    states += steps
    if t.sum(state) == 0 or t.sum(state) > 2:
        print("ISSUE DETECTED, STOP SIM",t.sum(state))
        break

imgs = [Image.fromarray((1-state)*255) for state in states]
imgs[0].save("t_1_reflective.gif", save_all=True, append_images=imgs[1:], duration=10, loop=100)

-------- MCS 0 --------------
-------------- Set 0 ---------------------
coordinates of the source pixels on the grid: 
 tensor([[0, 0],
        [0, 5],
        [4, 0],
        [4, 5]])
source pixels: tensor([0., 0., 0., 0.])
coordinates of the target pixels on the grid: 
 tensor([[0, 1],
        [1, 5],
        [3, 1],
        [5, 6]])
target pixels: tensor([0., 0., 0., 0.])
vol changes tensor([0., 0., 0., 0.])
all source IDs equivalent to target IDs
-------------- Set 1 ---------------------
coordinates of the source pixels on the grid: 
 tensor([[0, 2],
        [0, 6],
        [4, 3],
        [5, 6]])
source pixels: tensor([0., 0., 0., 0.])
coordinates of the target pixels on the grid: 
 tensor([[1, 2],
        [0, 5],
        [3, 3],
        [4, 5]])
target pixels: tensor([0., 0., 0., 0.])
vol changes tensor([0., 0., 0., 0.])
all source IDs equivalent to target IDs
-------------- Set 2 ---------------------
coordinates of the source pixels on the grid: 
 tensor([[3, 0],
        [3,

In [4]:
col_idxs = t.arange(8.0).repeat(8, 1) + 1
row_idxs = t.arange(8.0).repeat(8, 1).T + 1

x_mean = t.sum(init_state * col_idxs) / t.sum(init_state)
y_mean = t.sum(init_state * row_idxs) / t.sum(init_state)

print(x_mean, y_mean)

x_mean_adjusted = t.sum(state * col_idxs) / t.sum(state)
y_mean_adjusted = t.sum(state * row_idxs) / t.sum(state)

print(x_mean_adjusted, y_mean_adjusted)

dist = t.sqrt((x_mean - x_mean_adjusted)**2 + (y_mean-y_mean_adjusted)**2)

print(dist)


tensor(5.) tensor(5.)
tensor(5., grad_fn=<DivBackward0>) tensor(3., grad_fn=<DivBackward0>)
tensor(2., grad_fn=<SqrtBackward0>)


In [5]:
t.autograd.grad(dist, temperature)

c:\Users\jsche\anaconda3\envs\torchenv\lib\site-packages\torch\autograd\__init__.py:300: UserWarning: Error detected in ReflectionPad2DBackward0. Traceback of forward call that caused the error:
  File "c:\Users\jsche\anaconda3\envs\torchenv\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\jsche\anaconda3\envs\torchenv\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\jsche\anaconda3\envs\torchenv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\jsche\anaconda3\envs\torchenv\lib\site-packages\traitlets\config\application.py", line 1041, in launch_instance
    app.start()
  File "c:\Users\jsche\anaconda3\envs\torchenv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "c:\Users\jsche\anaconda3\envs\torchenv\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
    self.asyncio_l

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 8, 8]], which is output 0 of AddBackward0, is at version 400; expected version 384 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!